In [83]:
import numpy as np
import os
import pandas as pd
import shutil
import nipype.interfaces.fsl as fsl
import matplotlib.pyplot as plt
from pathlib import Path
import nibabel as nib



Apply BET mask to the T1 brain registered to FLAIR space for consistency.

In [28]:
apply_mask_BET = fsl.ApplyMask()
apply_mask_BET.inputs.mask_file = '/home/ela/Documents/B-RAPIDD/B-RAP_0100/3D-FLAIR/brain_nifti/masks/B-RAP_0100_05_D1.nii'
apply_mask_BET.inputs.out_file = '/home/ela/Documents/B-RAPIDD/B-RAP_0100/3D-FLAIR/registered_T1/B-RAP_0100_01_E1_BET.nii.gz'
T1_bet_res = apply_mask_BET.run()
print('Extracted T1 brain')


ValueError: ApplyMask requires a value for input 'in_file'. For a list of required inputs, see ApplyMask.help()

After applying FAST, load in the mask for CSF and apply it to the chosen variance map.
(B-RAPP_0100 Affine variance)

In [ ]:
csf_mask = nib.load('/home/ela/Documents/B-RAPIDD/B-RAP_0100/3D-FLAIR/registered_T1/B-RAP_0100_01_E1_CSF_mask.nii.gz').get_fdata()
variance_map = nib.load('/home/ela/Documents/B-RAPIDD/B-RAP_0100/3D-FLAIR/variance_maps/affine/intrasubnormalised_all_timepoints.nii.gz').get_fdata()
out_file = '/home/ela/Documents/B-RAPIDD/B-RAP_0100/3D-FLAIR/variance_maps/affine/no_csf/intrasubnormalised_all_timepoints_no_csf.nii.gz'


# invert the mask to keep non-csf
inv_csf_mask = np.logical_not(csf_mask)*1 # * 1 converts to 1 or 0 from a true/false to allow multiplication

variance_map_no_csf = np.multiply(variance_map,inv_csf_mask) # element-wise multiplication

print(variance_map_no_csf.shape)

# save the map
var_no_csf_nifti =  nib.Nifti1Image(variance_map_no_csf, affine=np.eye(4))
nib.save(var_no_csf_nifti, out_file)

(176, 232, 256)


After running Z-score calculation on the CSF-free variance map and obtaining the cluster map, we need to analyse the clusters. This is for B-RAPP_0100 Affine Variance.

In [ ]:
# load in the cluster information CSV
B_RAPP_0100_var_cluster_info = pd.read_csv('/home/ela/Documents/B-RAPIDD/B-RAP_0100/3D-FLAIR/z_score_maps/clusters/intrasubject/variance_affine_no_csf_cluster_info.txt', delimiter='\t')
B_RAPP_0100_var_cluster_info.head()

,Cluster Index,Voxels,MAX,MAX X (vox),MAX Y (vox),MAX Z (vox),COG X (vox),COG Y (vox),COG Z (vox)
0,2838,14045,34.4,65,37,155,66.5,42.0,169
1,2837,8650,45.8,80,65,103,97.8,87.1,128
2,2836,3059,12.6,122,43,132,104.0,51.4,129
3,2835,2265,14.3,99,116,162,101.0,108.0,164
4,2834,1751,12.1,70,98,170,74.2,107.0,165


In [ ]:
# count number of clusters with more than 10 voxels
num_clusters = len(B_RAPP_0100_var_cluster_info[B_RAPP_0100_var_cluster_info['Voxels'] >= 10])
num_clusters

320

In [ ]:
# extract the cluster indices for the clusters with more than or equal to 10 voxels
big_clusters = B_RAPP_0100_var_cluster_info[B_RAPP_0100_var_cluster_info['Voxels'] >= 10]
big_clusters

,Cluster Index,Voxels,MAX,MAX X (vox),MAX Y (vox),MAX Z (vox),COG X (vox),COG Y (vox),COG Z (vox)
0,2838,14045,34.40,65,37,155,66.5,42.0,169
1,2837,8650,45.80,80,65,103,97.8,87.1,128
2,2836,3059,12.60,122,43,132,104.0,51.4,129
3,2835,2265,14.30,99,116,162,101.0,108.0,164
4,2834,1751,12.10,70,98,170,74.2,107.0,165
...,...,...,...,...,...,...,...,...,...
315,2523,10,3.97,133,103,117,132.0,103.0,117
316,2522,10,2.66,65,89,146,65.3,89.9,145
317,2521,10,3.57,112,38,159,112.0,36.5,158
318,2520,10,4.34,68,64,197,68.1,62.8,198


In [ ]:
# these cluster indices are equal to the intensity values of the cluster in the variance map. Therefore, we want to select the voxels in the variance map that have these intensity values. 
# this ensures we only retain the biggest clusters.
# load in the clustered z-score variance map without csf
B_RAPP_0100_var_cluster_map = nib.load('/home/ela/Documents/B-RAPIDD/B-RAP_0100/3D-FLAIR/z_score_maps/clusters/intrasubject/variance_affine_no_csf.nii.gz').get_fdata()
# verify the map intensity values to see if it matches the pd dataframe by extracting max value
np.max(B_RAPP_0100_var_cluster_map)

2838.0

In [30]:
np.min(big_clusters['Cluster Index'].tolist()) # this is the threshold of the cluster indices such that the higher indices > 10

2519

In [36]:
# threshold the cluster map
thresh = fsl.Threshold()
thresh.inputs.in_file = '/home/ela/Documents/B-RAPIDD/B-RAP_0100/3D-FLAIR/z_score_maps/clusters/intrasubject/variance_affine_no_csf.nii.gz'
thresh.inputs.thresh = np.min(big_clusters['Cluster Index'].tolist()) 
thresh.inputs.direction = 'below'
thresh.inputs.out_file = '/home/ela/Documents/B-RAPIDD/B-RAP_0100/3D-FLAIR/z_score_maps/clusters/intrasubject/variance_affine_no_csf_thresholded.nii.gz'
thresh.run()
print('Thresholded cluster map')

Thresholded cluster map


In [37]:
# read in thresholded cluster map and extract unique values for verification
B_RAPP_0100_var_cluster_map_thresh = nib.load('/home/ela/Documents/B-RAPIDD/B-RAP_0100/3D-FLAIR/z_score_maps/clusters/intrasubject/variance_affine_no_csf_thresholded.nii.gz').get_fdata()
print(np.unique(B_RAPP_0100_var_cluster_map_thresh))

[   0. 2519. 2520. 2521. 2522. 2523. 2524. 2525. 2526. 2527. 2528. 2529.
 2530. 2531. 2532. 2533. 2534. 2535. 2536. 2537. 2538. 2539. 2540. 2541.
 2542. 2543. 2544. 2545. 2546. 2547. 2548. 2549. 2550. 2551. 2552. 2553.
 2554. 2555. 2556. 2557. 2558. 2559. 2560. 2561. 2562. 2563. 2564. 2565.
 2566. 2567. 2568. 2569. 2570. 2571. 2572. 2573. 2574. 2575. 2576. 2577.
 2578. 2579. 2580. 2581. 2582. 2583. 2584. 2585. 2586. 2587. 2588. 2589.
 2590. 2591. 2592. 2593. 2594. 2595. 2596. 2597. 2598. 2599. 2600. 2601.
 2602. 2603. 2604. 2605. 2606. 2607. 2608. 2609. 2610. 2611. 2612. 2613.
 2614. 2615. 2616. 2617. 2618. 2619. 2620. 2621. 2622. 2623. 2624. 2625.
 2626. 2627. 2628. 2629. 2630. 2631. 2632. 2633. 2634. 2635. 2636. 2637.
 2638. 2639. 2640. 2641. 2642. 2643. 2644. 2645. 2646. 2647. 2648. 2649.
 2650. 2651. 2652. 2653. 2654. 2655. 2656. 2657. 2658. 2659. 2660. 2661.
 2662. 2663. 2664. 2665. 2666. 2667. 2668. 2669. 2670. 2671. 2672. 2673.
 2674. 2675. 2676. 2677. 2678. 2679. 2680. 2681. 26

It's hard to see these thresholds because they're so close, so randomly allocate a mapping between cluster labels and a random value.

In [57]:
"""import random
min_intensity = 1
max_intensity = 3000

# Generate a list of unique random intensity values for each cluster
random_intensities = random.sample(range(min_intensity, max_intensity + 1), num_clusters+1)

# Now, random_intensities contains unique random values within the specified range for each cluster.
label_to_intensity = {label: intensity for label, intensity in zip(np.unique(B_RAPP_0100_var_cluster_map_thresh), random_intensities)}
label_to_intensity[0] = 0 # retain 0 value for background

# Map cluster labels to random intensities using the dictionary with a default value
mapped_image = np.vectorize(label_to_intensity.get, otypes=[int])(B_RAPP_0100_var_cluster_map_thresh)

# save the map
mapped_var_no_csf_nifti =  nib.Nifti1Image(mapped_image, affine=np.eye(4))
out_file = '/home/ela/Documents/B-RAPIDD/B-RAP_0100/3D-FLAIR/z_score_maps/clusters/intrasubject/variance_affine_no_csf_thresholded_mapped.nii.gz'
nib.save(mapped_var_no_csf_nifti, out_file)

"""

# Sort the unique cluster labels to retain the order
unique_labels = np.unique(B_RAPP_0100_var_cluster_map_thresh)
max_intensity = np.max(big_clusters['Cluster Index'].tolist())
min_intensity = np.min(big_clusters['Cluster Index'].tolist())

# Calculate the step size for intensity increments
step_size = 20

# Create a mapping dictionary based on the sorted labels
label_to_intensity = {label: min_intensity + idx * step_size for idx, label in enumerate(unique_labels)}
label_to_intensity[0] = 0 # retain 0 value for background

# Map cluster labels to intensity values using the dictionary
mapped_image = np.vectorize(label_to_intensity.get, otypes=[int])(B_RAPP_0100_var_cluster_map_thresh)

# save the map
mapped_var_no_csf_nifti =  nib.Nifti1Image(mapped_image, affine=np.eye(4))
out_file = '/home/ela/Documents/B-RAPIDD/B-RAP_0100/3D-FLAIR/z_score_maps/clusters/intrasubject/variance_affine_no_csf_thresholded_mapped.nii.gz'
nib.save(mapped_var_no_csf_nifti, out_file)

In [79]:
# visualise by cluster size

unique_labels = np.unique(B_RAPP_0100_var_cluster_map_thresh)

voxel_size = big_clusters['Voxels'].tolist()
voxel_size.reverse()
voxel_size.insert(0, 0) # for background

# extract voxels from big clusters

value_to_intensity = {original:vox_size for original, vox_size in zip(unique_labels, voxel_size)}
print(value_to_intensity)

# Map cluster labels to intensity values using the dictionary
vox_mapped_image = np.vectorize(value_to_intensity.get, otypes=[int])(B_RAPP_0100_var_cluster_map_thresh)

# save the map
mapped_vox_size_var_no_csf_nifti =  nib.Nifti1Image(vox_mapped_image, affine=np.eye(4))
out_file = '/home/ela/Documents/B-RAPIDD/B-RAP_0100/3D-FLAIR/z_score_maps/clusters/intrasubject/variance_affine_no_csf_thresholded_vox_size.nii.gz'
nib.save(mapped_vox_size_var_no_csf_nifti, out_file)

{0.0: 0, 2519.0: 10, 2520.0: 10, 2521.0: 10, 2522.0: 10, 2523.0: 10, 2524.0: 10, 2525.0: 10, 2526.0: 10, 2527.0: 10, 2528.0: 10, 2529.0: 10, 2530.0: 10, 2531.0: 10, 2532.0: 10, 2533.0: 10, 2534.0: 10, 2535.0: 10, 2536.0: 10, 2537.0: 10, 2538.0: 10, 2539.0: 11, 2540.0: 11, 2541.0: 11, 2542.0: 11, 2543.0: 11, 2544.0: 11, 2545.0: 11, 2546.0: 11, 2547.0: 11, 2548.0: 11, 2549.0: 11, 2550.0: 11, 2551.0: 11, 2552.0: 11, 2553.0: 11, 2554.0: 11, 2555.0: 11, 2556.0: 11, 2557.0: 11, 2558.0: 12, 2559.0: 12, 2560.0: 12, 2561.0: 12, 2562.0: 12, 2563.0: 12, 2564.0: 12, 2565.0: 12, 2566.0: 12, 2567.0: 12, 2568.0: 12, 2569.0: 12, 2570.0: 12, 2571.0: 12, 2572.0: 12, 2573.0: 13, 2574.0: 13, 2575.0: 13, 2576.0: 13, 2577.0: 13, 2578.0: 13, 2579.0: 13, 2580.0: 13, 2581.0: 13, 2582.0: 13, 2583.0: 13, 2584.0: 13, 2585.0: 14, 2586.0: 14, 2587.0: 14, 2588.0: 14, 2589.0: 14, 2590.0: 14, 2591.0: 14, 2592.0: 14, 2593.0: 14, 2594.0: 14, 2595.0: 14, 2596.0: 14, 2597.0: 15, 2598.0: 15, 2599.0: 15, 2600.0: 15, 2601.0:

In [81]:
# look at csf ARIA maps
csf_mask = nib.load('/home/ela/Documents/B-RAPIDD/B-RAP_0100/3D-FLAIR/registered_T1/B-RAP_0100_01_E1_CSF_mask.nii.gz').get_fdata()
variance_map = nib.load('/home/ela/Documents/B-RAPIDD/B-RAP_0100/3D-FLAIR/variance_maps/affine/intrasubnormalised_all_timepoints.nii.gz').get_fdata()
out_file = '/home/ela/Documents/B-RAPIDD/B-RAP_0100/3D-FLAIR/variance_maps/affine/no_csf/intrasubnormalised_all_timepoints_csf_only.nii.gz'


variance_map_only_csf = np.multiply(variance_map,csf_mask) # element-wise multiplication

print(variance_map_only_csf.shape)

# save the map
var_only_csf_nifti =  nib.Nifti1Image(variance_map_only_csf, affine=np.eye(4))
nib.save(var_only_csf_nifti, out_file)

(176, 232, 256)


In [82]:
# load in the cluster information CSV
B_RAPP_0100_var_cluster_info = pd.read_csv('/home/ela/Documents/B-RAPIDD/B-RAP_0100/3D-FLAIR/z_score_maps/clusters/intrasubject/variance_affine_csf_cluster_info.txt', delimiter='\t')
B_RAPP_0100_var_cluster_info.head()

,Cluster Index,Voxels,MAX,MAX X (vox),MAX Y (vox),MAX Z (vox),COG X (vox),COG Y (vox),COG Z (vox)
0,2153,19315,59.70,89,66,101,91.2,89.6,120
1,2152,2894,27.90,59,39,178,66.4,35.1,178
2,2151,2475,5.72,134,118,193,121.0,137.0,186
3,2150,2470,33.10,66,39,158,63.3,55.6,163
4,2149,1797,5.60,117,115,209,105.0,119.0,207


In [85]:
# count number of clusters with more than 10 voxels
num_clusters = len(B_RAPP_0100_var_cluster_info[B_RAPP_0100_var_cluster_info['Voxels'] >= 10])

# extract the cluster indices for the clusters with more than or equal to 10 voxels
big_clusters = B_RAPP_0100_var_cluster_info[B_RAPP_0100_var_cluster_info['Voxels'] >= 10]

# these cluster indices are equal to the intensity values of the cluster in the variance map. Therefore, we want to select the voxels in the variance map that have these intensity values. 
# this ensures we only retain the biggest clusters.
# load in the clustered z-score variance map without csf
B_RAPP_0100_var_cluster_map = nib.load('/home/ela/Documents/B-RAPIDD/B-RAP_0100/3D-FLAIR/z_score_maps/clusters/intrasubject/variance_affine_no_csf.nii.gz').get_fdata()
# verify the map intensity values to see if it matches the pd dataframe by extracting max value
np.max(B_RAPP_0100_var_cluster_map)

# threshold the cluster map
thresh = fsl.Threshold()
thresh.inputs.in_file = '/home/ela/Documents/B-RAPIDD/B-RAP_0100/3D-FLAIR/z_score_maps/clusters/intrasubject/variance_affine_csf.nii.gz'
thresh.inputs.thresh = np.min(big_clusters['Cluster Index'].tolist()) 
thresh.inputs.direction = 'below'
thresh.inputs.out_file = '/home/ela/Documents/B-RAPIDD/B-RAP_0100/3D-FLAIR/z_score_maps/clusters/intrasubject/variance_affine_csf_thresholded.nii.gz'
thresh.run()
print('Thresholded cluster map')

# visualise by cluster size

# read in thresholded cluster map and extract unique values for verification
B_RAPP_0100_var_cluster_map_thresh = nib.load('/home/ela/Documents/B-RAPIDD/B-RAP_0100/3D-FLAIR/z_score_maps/clusters/intrasubject/variance_affine_csf_thresholded.nii.gz').get_fdata()

unique_labels = np.unique(B_RAPP_0100_var_cluster_map_thresh)

voxel_size = big_clusters['Voxels'].tolist()
voxel_size.reverse()
voxel_size.insert(0, 0) # for background

# extract voxels from big clusters

value_to_intensity = {original:vox_size for original, vox_size in zip(unique_labels, voxel_size)}
print(value_to_intensity)

# Map cluster labels to intensity values using the dictionary
vox_mapped_image = np.vectorize(value_to_intensity.get, otypes=[int])(B_RAPP_0100_var_cluster_map_thresh)

# save the map
mapped_vox_size_var_no_csf_nifti =  nib.Nifti1Image(vox_mapped_image, affine=np.eye(4))
out_file = '/home/ela/Documents/B-RAPIDD/B-RAP_0100/3D-FLAIR/z_score_maps/clusters/intrasubject/variance_affine_csf_thresholded_vox_size.nii.gz'
nib.save(mapped_vox_size_var_no_csf_nifti, out_file)

Thresholded cluster map
{0.0: 0, 1901.0: 10, 1902.0: 10, 1903.0: 10, 1904.0: 10, 1905.0: 10, 1906.0: 10, 1907.0: 10, 1908.0: 10, 1909.0: 10, 1910.0: 10, 1911.0: 10, 1912.0: 10, 1913.0: 10, 1914.0: 10, 1915.0: 10, 1916.0: 10, 1917.0: 10, 1918.0: 10, 1919.0: 10, 1920.0: 11, 1921.0: 11, 1922.0: 11, 1923.0: 11, 1924.0: 11, 1925.0: 11, 1926.0: 11, 1927.0: 11, 1928.0: 11, 1929.0: 11, 1930.0: 11, 1931.0: 12, 1932.0: 12, 1933.0: 12, 1934.0: 12, 1935.0: 12, 1936.0: 12, 1937.0: 12, 1938.0: 12, 1939.0: 12, 1940.0: 12, 1941.0: 12, 1942.0: 12, 1943.0: 12, 1944.0: 12, 1945.0: 13, 1946.0: 13, 1947.0: 13, 1948.0: 13, 1949.0: 13, 1950.0: 13, 1951.0: 13, 1952.0: 13, 1953.0: 13, 1954.0: 13, 1955.0: 13, 1956.0: 14, 1957.0: 14, 1958.0: 14, 1959.0: 14, 1960.0: 14, 1961.0: 14, 1962.0: 14, 1963.0: 14, 1964.0: 15, 1965.0: 15, 1966.0: 15, 1967.0: 15, 1968.0: 15, 1969.0: 15, 1970.0: 15, 1971.0: 16, 1972.0: 16, 1973.0: 16, 1974.0: 16, 1975.0: 17, 1976.0: 17, 1977.0: 17, 1978.0: 17, 1979.0: 18, 1980.0: 18, 1981.0: